In [4]:
import dask.dataframe as dd

import pandas as pd
import numpy as np

from os.path import join
import os

# define path 
from dotenv import load_dotenv
load_dotenv()  

path = os.environ['PROJECT_PATH']

In [9]:
### import snapshot blocks heights 
df_snapshots = pd.read_csv("./assets/snapshot_selection.csv")
df_snapshots

,Nr.,Block Height,Date
0,1,7962629,2019-06-15
1,2,8155117,2019-07-15
2,3,8354625,2019-08-15
3,4,8553607,2019-09-15
4,5,8745378,2019-10-15
5,6,8938208,2019-11-15
6,7,9110216,2019-12-15
7,8,9285458,2020-01-15
8,9,9487426,2020-02-15
9,10,9676110,2020-03-15


In [5]:
ddf = dd.read_csv(join(path,'token_transfers_merged_to_15049989.csv')) #number of rows 1,622,714,622

In [6]:
ddf

,Unnamed: 0,token_address,from_address,to_address,value,transaction_hash,log_index,block_number
npartitions=121,,,,,,,,
,int64,object,object,object,object,object,int64,int64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [11]:
min_block = 4022304 #ds.min('block_number') 

In [12]:
max_block = 15049989 #ds.max('block_number')

In [13]:
ddf2 = ddf.set_index(ddf.block_number,sorted=True)

In [14]:
first_snapshot_height = 7962629 # Nadler & Schaer 

In [15]:
ddf3 = ddf2[ddf2.block_number <= first_snapshot_height]

In [16]:
ddf3.value = ddf3.value.astype('float64')

In [23]:
ddf3

,Unnamed: 0,token_address,from_address,to_address,value,transaction_hash,log_index,block_number
npartitions=121,,,,,,,,
4022304,int64,object,object,object,float64,object,int64,int64
4320872,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
14979534,...,...,...,...,...,...,...,...
15049989,...,...,...,...,...,...,...,...


In [52]:
ddf_outflows = ddf3.groupby(['token_address','from_address']).value.sum().compute().reset_index()
ddf_inflows = ddf3.groupby(['token_address', 'to_address',]).value.sum().compute().reset_index()

In [38]:
##convert to pandas

df_outflows = ddf_outflows.to_frame().reset_index()
df_inflows = ddf_inflows.to_frame().reset_index()

In [39]:
#format
df_outflows.value = df_outflows.value * -1
df_outflows.columns = ['token_address', 'address', 'value']
df_inflows.columns = ['token_address', 'address', 'value']

Index(['token_address', 'from_address', 'value'], dtype='object')

In [45]:
## merged
df_all = pd.concat([df_inflows,df_outflows]) 


In [48]:
df_token_holder_snapshot = df_all.groupby(['address','token_address']).value.sum()

In [56]:
### put into script ###

# import 
import dask.dataframe as dd

import pandas as pd
import numpy as np

from os.path import join
import os

# define path 
from dotenv import load_dotenv
load_dotenv()  

path = os.environ['PROJECT_PATH']


# load all token transfers
ddf = dd.read_csv(join(path,'token_transfers_merged_to_15049989.csv')) ##amend file 

# set value as int
ddf.value = ddf.value.astype('float64') 

# load all snapshots blockheights
df_snapshots = pd.read_csv("./assets/snapshot_selection.csv")

# iterate over snapshots 
for snapshot_block_height in df_snapshots['Block Height']:

    print(f'Snapshot Block Height: {snapshot_block_height}') 

    # filter df to snapshot height
    ddf_snapshot = ddf[ddf.block_number <= snapshot_block_height]

    # compute sums
    ddf_outflows = ddf_snapshot.groupby(['token_address','from_address']).value.sum().compute().reset_index()
    ddf_inflows = ddf_snapshot.groupby(['token_address', 'to_address',]).value.sum().compute().reset_index()  

    # format
    ddf_outflows.value = df_outflows.value * -1
    ddf_outflows.columns = ['token_address', 'address', 'value']
    ddf_inflows.columns = ['token_address', 'address', 'value']

    # stack df's
    df_all = pd.concat([df_inflows,df_outflows]) 

    # sum 
    df_token_holder_balance_snapshot = df_all.groupby(['address','token_address']).value.sum()

    # store tokenholder df as csv file
    df_token_holder_snapshot.to_csv(join(path,f'token_balance_lookup_tables/token_holder_snapshot_balance_{snapshot_block_height}.csv'))




Snapshot Block Height: 7962629
Snapshot Block Height: 8155117
